# Hyperparameter tuning (multi-node with Dask)

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="300">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll load data into a [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) and use [`dask-ml`](https://ml.dask.org/) to answer this question

> based on characteristics that can be known at the beginning of a trip, what tip will this trip earn (as a % of the total fare)?

<hr>

## Initialize A Dask Cluster

This tutorial uses multiple machines to show how to apply more computing resources to machine learning training. This is done with Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='large'` --> each worker machine will have 2 CPU cores and 16GB of RAM

To see a list of possible sizes, run the code below.

In [ ]:
import dask_saturn

dask_saturn.describe_sizes()

The `dask-saturn` code below creates two important objects: a cluster and a client.

* `cluster`: knows about and manages the scheduler and workers
    - can be used to create, resize, reconfigure, or destroy those resources
    - knows how to communicate with the scheduler, and where to find logs and diagnostic dashboards
* `client`: tells the cluster to do things
    - can send work to the cluster
    - can restart all the worker processes
    - can send data to the cluster or pull data back from the cluster

In [ ]:
import dask
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers,
    scheduler_size="medium",
    worker_size="large",
)
client = Client(cluster)

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple machines for data science workflows. To prove to yourself that Dask is taking advantage of the resources in the cluster, it's important to understand how to monitor that utilization while your code is running.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [ ]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

<hr>

## Connect to Snowflake

This example uses data stored in a Snowflake data warehouse that is managed by the team at Saturn Cloud. We've set up a read-only user for use in these examples. If you would like to access data stored in your own Snowflake account, you should set up [Credentials](https://www.saturncloud.io/docs/concepts/credentials/) for your account, user, and password then set the other connection information accordingly. For more details on Snowflake connection information, see ["Connecting to Snowflake"](https://docs.snowflake.com/en/user-guide/python-connector-example.html#connecting-to-snowflake) in the `snowflake-connector-python` docs.

Note that in order to update environment variables your Jupyter server will need to be stopped.

In [ ]:
import os
import snowflake.connector
import pandas as pd

conn_info = {
    "account": os.environ["EXAMPLE_SNOWFLAKE_ACCOUNT"],
    "user": os.environ["EXAMPLE_SNOWFLAKE_USER"],
    "password": os.environ["EXAMPLE_SNOWFLAKE_PASSWORD"],
    "database": os.environ["TAXI_DATABASE"],
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

This example uses Snowflake to handle the hard work of creating new features, then creates a Dask DataFrame with the result. It uses `dask.delayed()` to load partitions of the query result in parallel on the worker, so that you don't have to load the entire query result in this Jupyter and then send it to the Dask cluster.

The `load()` function specifies how each partition of the Dask dataframe should load its chunk of the data. As each partition of a Dask dataframe is a Pandas dataframe, we utilize the `pd.read_sql()` method.  We use standard python string formatting so that we can pass different values at execution time. In this case, we pull one day of taxi rides into each partition.

In [ ]:
query = """
SELECT * FROM (
    SELECT
        pickup_taxizone_id,
        dropoff_taxizone_id,
        passenger_count,
        DIV0(tip_amount, fare_amount) AS tip_fraction,
        DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
        WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
        HOUR(pickup_datetime) AS pickup_hour,
        (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
        MINUTE(pickup_datetime) AS pickup_minute
    FROM taxi_yellow
    WHERE
        DATE(pickup_datetime) = '{day}'
) SAMPLE (1)
"""


@dask.delayed
def load(conn_info, query, day):
    with snowflake.connector.connect(**conn_info) as conn:
        taxi = pd.read_sql(query.format(day=day), conn)
        taxi.columns = taxi.columns.str.lower()
        taxi = taxi.astype(float).fillna(-1)
        return taxi

Run a query to determine which specific dates we need to pull

In [ ]:
def get_dates(start, end):
    date_query = f"""
    SELECT
        DISTINCT(DATE(pickup_datetime)) as date
    FROM taxi_yellow
    WHERE
        pickup_datetime BETWEEN '{start}' and '{end}'
    """

    dates_df = pd.read_sql(date_query, conn)
    return dates_df["DATE"].tolist()


dates = get_dates("2019-01-01", "2019-01-31")

The `dd.from_delayed` function takes delayed Pandas outputs from a list and converts that to a high-level DataFrame object.

Since we're sampling in the Snowflake query, we run `persist()` to tell Dask to load all results into memory across the cluster. This ensures that Dask's lazy evaluation doesn't call the Snowflake query multiple times.

In [ ]:
import dask.dataframe as dd

taxi = dd.from_delayed([load(conn_info, query, day) for day in dates])
taxi = taxi.persist()
_ = wait(taxi)

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB")

In [ ]:
numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "tip_fraction"

taxi_train = taxi[features + [y_col]]

In [ ]:
taxi_train.head()

<hr>

## Run grid search

Now that you've loaded and preprocessed your training data, use the code below to find the best set of hyperparameters for your mode. This is done with `dask-ml`'s `GridSearchCV` and `scikit-learn`'s `ElasticNet`.

In [ ]:
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from dask_ml.compose import ColumnTransformer
from dask_ml.preprocessing import StandardScaler, DummyEncoder, Categorizer
from dask_ml.model_selection import GridSearchCV

pipeline = Pipeline(
    steps=[
        ("categorize", Categorizer(columns=categorical_feat)),
        ("onehot", DummyEncoder(columns=categorical_feat)),
        (
            "scale",
            ColumnTransformer(
                transformers=[("num", StandardScaler(), numeric_feat)],
                remainder="passthrough",
            ),
        ),
        ("clf", ElasticNet(normalize=False, max_iter=100)),
    ]
)

params = {
    "clf__l1_ratio": np.arange(0, 1.1, 0.1),
    "clf__alpha": [0, 0.5, 1, 2],
}

grid_search = GridSearchCV(pipeline, params, cv=3, scoring="neg_mean_squared_error")

Open up the Dask dashboard after you run the cell below, you'll see the grid search in action!

In [ ]:
%%time
_ = grid_search.fit(taxi_train[features], taxi_train[y_col])
grid_search.best_score_

In [ ]:
grid_search.best_params_

<hr>

## Save model

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

`GridSearchCV` automatically fits the best parameters to the full data and stores in `best_estimator_`

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/elastic_net_dask.pkl", "wb") as f:
    cloudpickle.dump(grid_search.best_estimator_, f)

<hr>

## Calculate metrics on test set

Use a different month for test set.

In [ ]:
test_dates = get_dates("2019-02-01", "2019-02-28")
taxi_test = dd.from_delayed([load(conn_info, query, day) for day in test_dates])
taxi_test = taxi_test.persist()
_ = wait(taxi_test)

`scikit-learn` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see ["Metrics and scoring"](https://scikit-learn.org/stable/modules/model_evaluation.html) in the `sciki-learn` docs.

This tutorial uses the `mean_squared_error` to evaluate the model. This metric penalizes large errors more than small errors.

In [ ]:
from sklearn.metrics import mean_squared_error

preds = grid_search.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)

<hr>

## Next Steps

In this tutorial, you learned how to use `dask-ml` to perform feature engineering and evaluate machine learning experiments based on distributed data from Snowflake in Dask DataFrames.

Next, try [this Dask + xgboost notebook](./xgboost-dask.ipynb) to see how to perform distributed XGBoost training using Dask.

Alternatively, try [this notebook](./dashboard.ipynb) to learn how to use Saturn Cloud to deploy an interactive dashboard that includes a component which uses the models trained above.

If you want to try working with larger datasets, change the code in this notebook to use more data or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

<hr>